## STA/STC Receptive field models for LNP neurons

This notebook demonstrates spike-triggered average (STA) and spike triggered covaraiance (STC) models for neural responses. It uses the output of synthetic, Linear-Nonlinear Poisson (LNP) neurons as a data source.

Inspiration/code from:
1. _"Analysis of Neuronal Spike Trains, Deconstructed"_, by J. Aljadeff, B.J. Lansdell, A.L. Fairhall and D. Kleinfeld (2016) Neuron, 91 http://dx.doi.org/10.1016/j.neuron.2016.05.039 
2. _"Spike-triggered neural characterization"_, by Schwartz, O, Pillow, JW, Rust, NC, and Simoncelli, EP. (2006).  Journal of Vision, 6 [Code](http://pillowlab.princeton.edu/code_STC.html)

_SLH 2017_

In [1]:
import numpy as np
from scipy import linalg,signal

import matplotlib.pyplot as plt
import matplotlib as mpl
import pylab

%matplotlib inline

In [2]:
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

## Stimuli

$\mathbf{s_g}$ : gaussian white noise stimulus vector

$\mathbf{s_c}$ : correlated gaussian white noise generated by convolving white noise with exponential: $\mathbf{s_c} = \mathbf{s_g} \ast e^{-t/\tau} $